In [21]:
import pandas as pd

df = pd.read_csv('/Users/cao/Library/CloudStorage/Nutstore-caokai1073@gmail.com/Github/strimap-tools-uhlerlab/examples/tcrphla_train_set_example_all.csv')
df = df.sample(frac=0.02).reset_index(drop=True)
df = df[['cdr3a','cdr3b','Va','Ja','Vb','Jb','peptide','HLA','label']]
df.to_csv('tcrphla_train_set_example.csv', index=False)

df = pd.read_csv('/Users/cao/Library/CloudStorage/Nutstore-caokai1073@gmail.com/Github/strimap-tools-uhlerlab/examples/tcrphla_test_set_example_all.csv')
df = df.sample(frac=0.02).reset_index(drop=True)
df = df[['cdr3a','cdr3b','Va','Ja','Vb','Jb','peptide','HLA','label']]
df.to_csv('tcrphla_test_set_example.csv', index=False)

In [ ]:
import pandas as pd
import numpy as np
import torch
from main import StriMap_pHLA, StriMap_TCRpHLA
from utils import negative_sampling_phla
from sklearn.model_selection import StratifiedKFold

df = pd.read_csv('path/to/train_set.csv')

device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
df_train_folds, df_val_folds = [], []

phla_fold = 5 # number of folds used in pHLA model training
tcrphla_fold = 10 # number of folds used in TCR-pHLA model training

skf = StratifiedKFold(n_splits=tcrphla_fold, shuffle=True, random_state=42)

df_train_folds = []
df_val_folds = []

for i, (train_idx, val_idx) in enumerate(skf.split(df_train, df_train["label"])):
    
    df_train_fold = df_train.iloc[train_idx].reset_index(drop=True)
    df_val_fold   = df_train.iloc[val_idx].reset_index(drop=True)

    df_val_fold_neg = negative_sampling_phla(df_val_fold, random_state=i)
    df_val_fold = pd.concat([df_val_fold, df_val_fold_neg], axis=0).reset_index(drop=True)

    df_train_folds.append(df_train_fold)
    df_val_folds.append(df_val_fold)

pep_hla_system = StriMap_pHLA(device=device, cache_dir='cache')
pep_hla_system.prepare_embeddings(df.reset_index(drop=True))

tcr_phla_system = StriMap_TCRpHLA(
    pep_hla_system=pep_hla_system,
    pep_hla_params=[f'params/phla_model_fold{i}.pt' for i in range(phla_fold)], # load trained pHLA model parameters
    device=device,
    model_save_path=f'params/tcrphla_model.pt',
    resample_negatives=True,
)

tcr_phla_system.prepare_embeddings(pd.concat([df_train, df_AS], axis=0).reset_index(drop=True))

all_histories = tcr_phla_system.train_kfold(
    train_folds=[(df_train_fold, df_val_fold) for df_train_fold, df_val_fold in zip(df_train_folds, df_val_folds)],
    epochs=100
)

In [ ]:
import pandas as pd
import numpy as np
import torch
from main import StriMap_pHLA, StriMap_TCRpHLA, load_test_data

df_test = pd.read_csv('path/to/test_set.csv')

tcrphla_fold = 10
phla_fold = 5

device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
pep_hla_system = StriMap_pHLA(device=device, cache_dir='tcrphla_cache/pep_background')
pep_hla_system.prepare_embeddings(df_test)

tcr_phla_system = StriMap_TCRpHLA(
    pep_hla_system=pep_hla_system,
    pep_hla_params=[f'params/phla_model_fold{i}.pt' for i in range(phla_fold)],
    device=device,
    model_save_path=f'params/tcrphla_model.pt',
)

tcr_phla_system.prepare_embeddings(df_test)

pred, pep_feat, attn_dict = tcr_phla_system.predict(df_test, batch_size=128, return_probs=True, use_kfold=True, num_folds=tcrphla_fold)
df_test['predicted_score'] = pred
print(df_test.head())